In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('seaborn-bright')

import pickle
from IPython.display import display
%load_ext memory_profiler
%load_ext line_profiler

In [63]:
# pitfalls, groupby column with NaN values
df = pd.DataFrame({'val0': [1.1,1.1,2.2,2.2,3.3,3.3,3.3], 'val1': [1,1,2,2,3,3,3], 'val2': [np.nan,'A','B','A','B','C',np.nan]})
display(df)

def myf(df):
    df['new_column'] = 666
    return df
df.groupby('val2').apply(myf)

,val0,val1,val2
0,1.1,1,NaN
1,1.1,1,A
2,2.2,2,B
3,2.2,2,A
4,3.3,3,B
5,3.3,3,C
6,3.3,3,NaN


,val0,val1,val2,new_column
0,NaN,NaN,NaN,NaN
1,1.1,1.0,A,666.0
2,2.2,2.0,B,666.0
3,2.2,2.0,A,666.0
4,3.3,3.0,B,666.0
5,3.3,3.0,C,666.0
6,NaN,NaN,NaN,NaN


In [58]:
# pitfalls, silent type convertion due to NaN
# http://stackoverflow.com/questions/36620256/why-is-pandas-converting-datetimes-to-float-in-aggregate-function
# https://github.com/pydata/pandas/issues/12941

df = pd.DataFrame(np.random.rand(250).reshape(50,5), 
                      index=pd.date_range('1/1/2016', periods=50, freq='H' ),
                      columns=list('ABCDE'))
df['timestamp'] = df.index

print(df.timestamp.resample('30min',label='right').last().head(3))
print('==========')
print(df.timestamp.resample('2H',label='right').last().head(3))


2016-01-01 00:30:00    1.451606e+18
2016-01-01 01:00:00             NaN
2016-01-01 01:30:00    1.451610e+18
Freq: 30T, Name: timestamp, dtype: float64
2016-01-01 02:00:00   2016-01-01 01:00:00
2016-01-01 04:00:00   2016-01-01 03:00:00
2016-01-01 06:00:00   2016-01-01 05:00:00
Freq: 2H, Name: timestamp, dtype: datetime64[ns]
